In [1]:
# test 1
def calculate_category_distribution(transit, residential, commercial, attraction):
    # Define weights for how each input factor affects each category
    weights = {
        'Beverage': {'transit': 0.6, 'residential': 0.3, 'commercial': 0.7, 'attraction': 0.7},
        'Grab & Go': {'transit': 0.3, 'residential': 0.1, 'commercial': 0.7, 'attraction': 0.7},
        'Frozen Food': {'transit': 0.1, 'residential': 0.8, 'commercial': 0.3, 'attraction': 0.1},
        'Household Supplies': {'transit': 0.6, 'residential': 0.9, 'commercial': 0.1, 'attraction': 0.1}
    }

    # Normalize input factors
    total_input_score = transit + residential + commercial + attraction
    if total_input_score == 0:
        return "All input factors are zero; no distribution can be made."

    # Calculate raw weighted scores for each category
    categories = {}
    for category, factor_weights in weights.items():
        categories[category] = (
            factor_weights['transit'] * transit +
            factor_weights['residential'] * residential +
            factor_weights['commercial'] * commercial +
            factor_weights['attraction'] * attraction
        )

    # Normalize the raw scores to sum to 100%
    total_category_score = sum(categories.values())
    for category in categories:
        categories[category] = (categories[category] / total_category_score) * 100

    # Ensure that each category gets at least 10%
    min_allocation = 10
    adjustment_needed = 0
    adjusted_categories = {}

    # Identify categories that are below 10%
    for category in categories:
        if categories[category] < min_allocation:
            adjustment_needed += min_allocation - categories[category]
            adjusted_categories[category] = min_allocation
        else:
            adjusted_categories[category] = categories[category]

    # Redistribute the excess space proportionally to the categories that are above 10%
    total_adjustable = sum([adjusted_categories[category] for category in adjusted_categories if adjusted_categories[category] > min_allocation])
    remaining_space = 100 - sum(adjusted_categories.values())

    if remaining_space > 0 and total_adjustable > 0:
        scale_factor = remaining_space / total_adjustable
        for category in adjusted_categories:
            if adjusted_categories[category] > min_allocation:
                adjusted_categories[category] += (adjusted_categories[category] - min_allocation) * scale_factor

    return adjusted_categories


In [2]:
transit = 3
residential = 4
commercial = 2
attraction = 1

distribution = calculate_category_distribution(transit, residential, commercial, attraction)
print(distribution)


{'Beverage': 27.71739130434783, 'Grab & Go': 18.478260869565215, 'Frozen Food': 22.82608695652174, 'Household Supplies': 30.978260869565222}


In [1]:
def calculate_layout_with_services(transit, residential, commercial, attraction):
    # Define weights for category allocation
    category_weights = {
        'Beverage': {'transit': 0.6, 'residential': 0.3, 'commercial': 0.7, 'attraction': 0.7},
        'Grab & Go': {'transit': 0.3, 'residential': 0.1, 'commercial': 0.7, 'attraction': 0.7},
        'Frozen Food': {'transit': 0.1, 'residential': 0.8, 'commercial': 0.3, 'attraction': 0.1},
        'Household Supplies': {'transit': 0.6, 'residential': 0.9, 'commercial': 0.1, 'attraction': 0.1}
    }
    
    # Optional services with weights
    services_weights = {
        'Pick-up lockers': {'transit': 0.8, 'residential': 0.9, 'commercial': 0.3, 'attraction': 0.1},
        'Coffee service': {'transit': 0.4, 'residential': 0.7, 'commercial': 0.8, 'attraction': 0.1},
        'Seats': {'transit': 0.3, 'residential': 0.1, 'commercial': 0.7, 'attraction': 0.8},
        'Self-checkout kiosk': {'commercial': 0.5, 'transit': 0.5},
        'Lunch stands': {'transit': 0.3, 'residential': 0.7, 'commercial': 0.8, 'attraction': 0.3},
        'Bakery goods': {'transit': 0.3, 'residential': 0.8, 'commercial': 0.6, 'attraction': 0.2},
        'Fresh sweet treats': {'transit': 0.6, 'residential': 0.8, 'commercial': 0.3, 'attraction': 0.8}
    }

    # Area assumptions for services if added
    service_area_requirements = {
        'Pick-up lockers': 5,
        'Coffee service': 5,
        'Seats': 5,
        'Self-checkout kiosk': 6,
        'Lunch stands': 10,
        'Bakery goods': 3,
        'Fresh sweet treats': 10
    }

# Calculate initial category scores based on weights
    category_scores = {}
    for category, factors in category_weights.items():
        category_scores[category] = (
            factors['transit'] * transit + 
            factors['residential'] * residential + 
            factors['commercial'] * commercial + 
            factors['attraction'] * attraction
        )
    
    # Normalize category scores to sum to 100%
    total_score = sum(category_scores.values())
    for category in category_scores:
        category_scores[category] = (category_scores[category] / total_score) * 100

    # Calculate scores for optional services
    service_scores = {}
    for service, factors in services_weights.items():
        service_scores[service] = sum(factors[factor] * eval(factor) for factor in factors)
    
    # Determine which services to add based on score > 5
    added_services = {}
    total_service_area = 0
    max_service_area = 30  # Max 30% of layout area for services

    for service, score in service_scores.items():
        if score > 5 and (total_service_area + service_area_requirements[service]) <= max_service_area:
            added_services[service] = service_area_requirements[service]
            total_service_area += service_area_requirements[service]
    
    # Adjust category areas to fit within the remaining layout space
    remaining_area_for_categories = 100 - total_service_area
    adjusted_categories = {}
    for category in category_scores:
        adjusted_categories[category] = (category_scores[category] / 100) * remaining_area_for_categories

    # Combine adjusted categories and added services
    layout_allocation = {'Categories': adjusted_categories, 'Added Services': added_services}
    
    return layout_allocation

In [3]:
# TIMES SQUARE
transit = 3
residential = 1
commercial = 4
attraction = 4

layout = calculate_layout_with_services(transit, residential, commercial, attraction)
print(layout)

{'Categories': {'Beverage': 26.292682926829265, 'Grab & Go': 22.53658536585366, 'Frozen Food': 9.21951219512195, 'Household Supplies': 11.95121951219512}, 'Added Services': {'Coffee service': 5, 'Seats': 5, 'Lunch stands': 10, 'Fresh sweet treats': 10}}


In [2]:
# WASHINGTON SQUARE PARK
transit = 1
residential = 4
commercial = 2
attraction = 1

layout = calculate_layout_with_services(transit, residential, commercial, attraction)
print(layout)

{'Categories': {'Beverage': 21.809210526315788, 'Grab & Go': 15.657894736842104, 'Frozen Food': 22.36842105263158, 'Household Supplies': 25.164473684210527}, 'Added Services': {'Pick-up lockers': 5, 'Fresh sweet treats': 10}}


In [6]:
# LIC
transit = 2
residential = 4
commercial = 2
attraction = 1

layout = calculate_layout_with_services(transit, residential, commercial, attraction)
print(layout)

{'Categories': {'Beverage': 20.625000000000004, 'Grab & Go': 14.208333333333334, 'Frozen Food': 18.791666666666668, 'Household Supplies': 23.375000000000004}, 'Added Services': {'Pick-up lockers': 5, 'Coffee service': 5, 'Lunch stands': 10, 'Bakery goods': 3}}
